In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [106]:
# get all itrs explored for some (dvfs, qps) pair
def list_itrs(dvfs, qps, app, rapl='135', run='0'):
    app_dir = app + '_' + run + '_rapl_' + rapl + '/'
    qps_dir = app_dir + qps + '_qps/'
    rdtsc_dir = qps_dir + 'linux_' + app + '_rdtsc_' + run + '_' + dvfs + '_' + rapl + '_' + qps +'/'
    itrs = []
    for file in os.listdir(rdtsc_dir):
        tags = file.split('_')
        itr = tags[1]
        itrs.append(itr)
    itrs = list(set(itrs))
    return itrs

In [107]:
from numpy.linalg import eig, eigvalsh

def get_merged_eigvals(dvfs, qps, itr, app, rapl='135', run='0'):
    
    app_dir = app + '_' + run + '_rapl_' + rapl + '/'
    corr_dir = app_dir + qps + '_merged_corr/'
    merged_counters_corr_dir = corr_dir + qps + '_' + dvfs + '_counters_merged_corr/'
    merged_non_counters_corr_dir = corr_dir + qps + '_' + dvfs + '_non_counters_merged_corr/'
    
    merged_counters_corr_file = merged_counters_corr_dir + dvfs + '_' + qps + '_' + itr + '_counters_merged_corr'
    merged_counters_corr = pd.read_csv(merged_counters_corr_file, index_col=0)
    merged_counter_vals, merged_vecs = eig(merged_counters_corr)
    for i in range(0, len(merged_counter_vals)):
        merged_counter_vals[i] = merged_counter_vals[i] / merged_counter_vals[0]
    merged_non_counters_corr_file = merged_non_counters_corr_dir + dvfs + '_' + qps + '_' + itr + '_non_counters_merged_corr'
    merged_non_counters_corr = pd.read_csv(merged_non_counters_corr_file, index_col=0)
    merged_non_counter_vals, merged_vecs = eig(merged_non_counters_corr)
    for i in range(0, len(merged_non_counter_vals)):
        merged_non_counter_vals[i] = merged_non_counter_vals[i] / merged_non_counter_vals[0]    
    merged_eigvals = np.concatenate([np.log(merged_counter_vals[1:4]), np.log(merged_non_counter_vals[1:4])])
    
    return merged_eigvals

def get_latencies(dvfs, qps, itr, app, rapl='135', run='0'):
    app_dir = app + '_' + run + '_rapl_' + rapl + '/'
    qps_dir = app_dir + qps + '_qps/'
    out_dir = qps_dir + 'linux_' + app + '_out_' + run + '_' + dvfs + '_' + rapl + '_' + qps +'/'
    out_file = out_dir + 'linux.mcd.out.0_' + str(itr) + '_' + dvfs + '_135_' + qps[:-1] + '000'
    with open(out_file, 'r') as f:
        lines = f.readlines()
    header = lines[0].rstrip('\n').split()
    read_lat = lines[1].rstrip('\n').split()
    lat = {'read': dict(zip(header[1:], [float(y) for y in read_lat[1:]]))}

    return lat['read']

# def get_joules(dvfs, qps, itr, app, rapl='135', run='0'):
#     app_dir = app + '_' + run + '_rapl_' + rapl + '/'
#     qps_dir = app_dir + qps + '_qps/'
#     joules_sum = df['joules_diff'].sum()
#     joules = {'joules_sum': joules_sum}
    
#     return joules

In [108]:
app = 'mcd'
run = '0'
rapl = '135'
#dvfs = '0xd00'
#qps = '200k'

In [109]:
app_dir = app + '_' + run + '_rapl_' + rapl + '/'

In [6]:
print(app_dir)
df = pd.DataFrame()
for qps in ['200k', '400k', '600k']:
    qps_dir = app_dir + qps + '_qps/'
    #print(qps_dir)
    for dvfs in ['0xd00', '0xf00', '0x1100', '0x1300', '0x1500', '0x1700', '0x1900', '0x1b00', '0x1d00']:
        logs_dir = qps_dir + 'linux_' + app + '_dmesg_' + run + '_' + dvfs + '_' + rapl + '_' + qps + '/'
        rdtsc_dir = qps_dir + 'linux_' + app + '_rdtsc_' + run + '_' + dvfs + '_' + rapl + '_' + qps +'/'
        out_dir = qps_dir + 'linux_' + app + '_out_' + run + '_' + dvfs + '_' + rapl + '_' + qps +'/'
        itrs = list_itrs(dvfs, qps, app)
        cores = []
        cores = !ls $logs_dir | cut -d '_' -f2 | sort | uniq
        #print(logs_dir)
        #print(rdtsc_dir)
        #print(out_dir)
        #print(f'cores: {cores}')
        #print(f'itrs: {itrs}')
        for itr in itrs:
            new_row = {}
            eigvals = get_merged_eigvals(dvfs, qps, itr, app)
            i = 0
            for val in eigvals:
                new_row['eig_' + str(i)] = val
                i += 1
            #print(eigvals)
            latencies = get_latencies(dvfs, qps, itr, app)
            new_row['read_99th'] = latencies['99th']
            new_row['dvfs'] = dvfs
            new_row['qps'] = qps
            new_row['itr'] = itr
            new_row['run'] = run
            new_row['rapl'] = rapl
            new_row['desc'] = tuple([dvfs, qps, itr, rapl, run])
            #print(latencies)
            df = df._append(new_row, ignore_index = True)            


mcd_0_rapl_135/


In [7]:
df

,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,read_99th,dvfs,qps,itr,run,rapl,desc
0,-1.253670,-2.110212,-7.959926,0.453684,-1.116067,-0.912379,515.1,0xd00,200k,400,0,135,"(0xd00, 200k, 400, 135, 0)"
1,-1.359995,-2.087443,-7.719068,0.436452,-1.061680,-0.854757,354.3,0xd00,200k,250,0,135,"(0xd00, 200k, 250, 135, 0)"
2,-1.237434,-1.993171,-7.831584,0.445934,-1.091343,-0.885904,401.8,0xd00,200k,300,0,135,"(0xd00, 200k, 300, 135, 0)"
3,-1.354018,-2.016180,-7.800136,0.432198,-1.036436,-0.835023,297.3,0xd00,200k,200,0,135,"(0xd00, 200k, 200, 135, 0)"
4,-2.554598,-4.737903,-11.051273,0.449865,-1.101121,-0.893846,449.1,0xd00,200k,350,0,135,"(0xd00, 200k, 350, 135, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,-0.656961,-1.480661,-16.515956,0.367597,-1.032669,-0.666575,102.6,0x1d00,600k,30,0,135,"(0x1d00, 600k, 30, 135, 0)"
320,-1.016204,-2.464969,-7.357614,0.419007,-1.061871,-0.735698,115.0,0x1d00,600k,50,0,135,"(0x1d00, 600k, 50, 135, 0)"
321,-3.630636,-4.103923,-13.812358,0.100888,-1.094681,-0.607762,123.3,0x1d00,600k,2,0,135,"(0x1d00, 600k, 2, 135, 0)"
322,-2.466959,-3.324817,-11.328518,0.404195,-1.047116,-0.707301,110.9,0x1d00,600k,40,0,135,"(0x1d00, 600k, 40, 135, 0)"


In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [9]:
eig_cols = [c for c in df if c.find('eig')>-1]
#eig_cols = list(eig_cols[1:4]) + list(eig_cols[5:8])
eig_cols

['eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5']

In [10]:
lat_col = 'read_99th'

In [110]:
scorer = r2_score

def run_exp(model,
            df,
            scale=True, 
            ratios=True, 
            log=False, 
            n_iter=10, 
            scorer=r2_score,
            plot=False):

    
    score_list, model_list = [], []
    train_desc_list, test_desc_list = [], []
    np.random.seed(12345)
    
    for i in range(n_iter): 
        
        #split into train and test sets
        df_train, df_test = train_test_split(df, train_size=0.7)
        train_desc_list.append(df_train['desc'])
        test_desc_list.append(df_test['desc'])
        
        if i==0: print(df_train.shape, df_test.shape)

        #define features to be eigenvalues
        df_train_feats = df_train[eig_cols].copy()
        df_test_feats = df_test[eig_cols].copy()
        #define latency column to be target
        df_train_y = df_train[lat_col].copy()
        df_test_y = df_test[lat_col].copy()

        feat_col_names = df_train_feats.columns

        model.fit(df_train_feats, df_train_y)

        pred_train = model.predict(df_train_feats)
        pred_test = model.predict(df_test_feats)

        labels_train = df_train_y
        labels_test = df_test_y
        
        train_score = scorer(pred_train, labels_train)
        test_score = scorer(pred_test, labels_test)
        score_list.append((train_score, test_score))
        
        # plot accuracy: prediction-vs-label for train and test points
        if plot:
            plt.figure(figsize=(7,4))
            plt.plot(labels_test, pred_test, 'p', label='test')
            plt.plot(labels_train, pred_train, 'p', label='train')
            plt.plot(labels_train, labels_train, '-')
            #plt.xlim((0, 700))
            #plt.ylim((0, 700))
            plt.legend()
            plt.grid()
            plt.xlabel('labels')
            plt.ylabel('preds')
            plt.title(f'train score = {train_score} test score = {test_score}')
            plt.show()
            plt.close()

        
        if True:
            print()
            print(f'ITERATION {i}')
            print()
            train_list = [list(desc) for desc in train_desc_list[i].values]
            train_dvfs = np.array(train_list)[:, 0]
            train_qps = np.array(train_list)[:, 1]
            train_itr = np.array(train_list)[:, 2]
            test_list = [list(desc) for desc in test_desc_list[i].values]
            test_dvfs = np.array(test_list)[:, 0]
            test_qps = np.array(test_list)[:, 1]
            test_itr = np.array(test_list)[:, 2]
            
#             train_dvfs_count = {}
#             train_dvfs_set = set(list(train_dvfs))
#             for dvfs in train_dvfs_set:
#                 train_dvfs_count[dvfs] = list(train_dvfs).count(dvfs)
#             print(train_dvfs_count)
            
            
            # plot distributions
            fig1, ax1 = plt.subplots(figsize = (7, 4))
            plt.title("DVFS Distribution")
            plt.xlabel("DVFS")
            plt.ylabel("# Samples")
            train1_per_bin, train1_bins, _ = ax1.hist(train_dvfs)
            test1_per_bin, test1_bins, _ = ax1.hist(test_dvfs, bins=train1_bins)
            plt.show()
            plt.close()
            
            fig2, ax2 = plt.subplots(figsize =(7, 4))
            plt.title("QPS Distribution")
            plt.xlabel("QPS")
            plt.ylabel("# Samples")
            train2_per_bin, train2_bins, _ = ax2.hist(train_qps)
            test2_per_bin, test2_bins, _ = ax2.hist(test_qps, bins=train2_bins)
            plt.show()
            plt.close()
            
            fig3, ax3 = plt.subplots(figsize =(7, 4))
            plt.title("ITR-Delay Distribution")
            plt.xlabel("ITR-Delay")
            plt.ylabel("# Samples")
            train3_per_bin, train3_bins, _ = ax3.hist(train_itr)
            test3_per_bin, test3_bins, _ = ax3.hist(test_itr, bins=train3_bins)
            plt.show()
            plt.close()
            
        test_80th = labels_test.quantile(0.75)
        test_20th = labels_test.quantile(0.25)
        train_80th = labels_train.quantile(0.80)
        train_20th = labels_train.quantile(0.20)

        j = 0
        for pred in pred_train:
            if ((pred - train_80th) > 200) or ((train_20th - pred) > 200):
                print(f'TRAIN OUTLIER: {train_desc_list[i].iloc[[j]].values}, \
                label = {labels_train.iloc[[j]].values}, pred = {pred}')
            j += 1
        
        print()
        
        j = 0
        for pred in pred_test:
            if ((pred - test_80th) > 100) or ((test_20th - pred) > 100):
                print(f'TEST OUTLIER: {test_desc_list[i].iloc[[j]].values}, \
                label = {labels_test.iloc[[j]].values}, pred = {pred}')
            j += 1
                
    return score_list, model, feat_col_names, train_desc_list, test_desc_list

##### for model in [RandomForestRegressor()]:
#LinearRegression(), Lasso(), Ridge(), DecisionTreeRegressor(), RandomForestRegressor():
    score_list, model, feat_col_names, train_desc, test_desc = run_exp(model,
                                                df,
                                                scale=False, 
                                                ratios=False, 
                                                log=False, 
                                                n_iter=10, 
                                                scorer=r2_score,
                                                plot=True) 

    score_mean, score_std = np.mean([i[1] for i in score_list]), np.std([i[1] for i in score_list]) #no log
    print(f'model={model} {score_mean - score_std:.3f}--{score_mean:.3f}--{score_mean + score_std:.3f}\n')
    print()


In [36]:
score_list

[(0.8880978422774453, 0.6725891238315463),
 (0.9402337190892235, 0.4136924944656075),
 (0.9633882410332373, -0.02795568583732999),
 (0.9687849855442596, 0.0013657712980992276),
 (0.9350500613017181, 0.7640197540340358)]